In [2]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [3]:
class DMF:
    
    def __init__(self, user_num,item_num):
        
        latent_features = 8
        
        user = Input(shape = (1,),dtype = 'int32')
        item = Input(shape = (1,),dtype = 'int32')
        
        #embedding user, item for GMF 
        
        gmf_user_embedding = Embedding(user_num, latent_features,input_length = user.shape[1])(user)
        gmf_user_embedding = Flatten()(gmf_user_embedding)
        
        gmf_item_embedding = Embedding(item_num, latent_features, input_length = item.shape[1])(item)
        gmf_item_embedding = Flatten()(gmf_item_embedding)
        
        #embedding user, item for MLP
        
        mlp_user_embedding = Embedding(user_num,32,input_length = user.shape[1])(user)
        mlp_user_embedding = Flatten()(mlp_user_embedding)
        
        mlp_item_embedding = Embedding(item_num, 32, input_length = item.shape[1])(item)
        mlp_item_embedding = Flatten()(mlp_item_embedding)
        
        #GMF layers
        gmf_mul = Multiply()([gmf_user_embedding,gmf_item_embedding])
        
        #MLP layers
        mlp_concat = Concatenate([mlp_user_embedding,mlp_item_embedding])
        mlp_dropout = Dropout(0.2)(mlp_concat)
        
        #1 Layer 
        mlp_layer_1 = Dense(units = 64, activation = 'relu',name = 'mlp_layer1')(mlp_dropout)
        mlp_dropout1 = Dropout(rate = 0.2 , name = 'mlp1d')(mlp_layer_1)
        mlp_batchnorm1 = BatchNormalization()(mlp_dropout1)
        
        # 2 Layer
        mlp_layer2 = Dense(units = 32, activation ='relu',name = 'mlp_layer2')(mlp_batchnorm1)
        mlp_dropout2 = Dropout(rate = 0.2, name = 'mlp2d')(mlp_layer2)
        mlp_batchnorm2 = BatchNormalization()(mlp_dropout2)
        
        # 3 layer
        mlp_layer3  = Dense(units = 16,activation = 'relu',name = 'mlp_layer3')(mlp_batchnorm2)
        
        # 4 Layer
        mlp_layer4 = Dense(units = 8 , activation = 'relu',name = 'mlp_layer4')(mlp_layer3)
        
        
        # merget GMF + MLP
        merge_vector = tf.keras.layers.concatenate([gmf_mul,mlp_layer4])
        
        outpur_layer = Dense(1, kernel_initalizer = 'lecun_uniform',name = 'output_layer')(merge_vector)
        
        self.model = Model([user,item],outpur_layer)
        self.model.compile(
            optimizer = 'adam',
            loss = 'binary_crossentropy'
        )
        
    def ModelDMF(self):
        model = self.model
        return model

    